## Análisis de Sentimientos de críticas de películas

Junto con Keras, viene un ejemplo imdb_lstm.py. Este ejercicio esta prácticamente basado en él.

Es un gran ejemplo del uso de las RNNs.  El conjunto de datos que se utilizará consta de críticas de películas generadas por usuarios, y una classificación indicando si le gustó, o no, basado en su rating asociado. 

Hay más información de este conjunto de datos en:

https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification

Como la comprensión del lenguaje escrito requiere "llevar cuenta" de todas las palabras en una oración, necesitamos una RNN para mantener una "memoria" de las palabras que pasaron antes, conforme va "leyendo" oraciones a lo largo del tiempo. 

En particular, se usarán unidades LSTM (Long Short-Term Memory) porque no es deseable "olvidar" palabras demasiado rápido...las palabras al inicio de una oración pueden afectar el significado de la misma grandemente.

Empezamos por la importación de lo que se requiere:

In [25]:
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

Ahora importar los datos para entrenamiento y prueba.  Para que sea más manejable, se especifica que se quieren solamente las 20,000 palabras más populares en el conjunto de datos. Por algún motivo, este conjunto tiene una relación de 50% entreno y 50% prueba. 

In [26]:
print('Cargando los datos...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=50000)

Cargando los datos...


In [27]:
len(X_train)

25000

In [28]:
len(X_test)

25000

A ver cómo son los datos, el primer elemento de entrenamiento debe ser una crítica de una película:

In [29]:
X_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 22665,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 21631,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 31050,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5

Esto no parece una crítica de una película!!!!  Resulta que la gente que preparó los datos ya hicieron algo de preparación previa de los datos.  Estos números coinciden con el índice correspondiente a cada palabra de la crítica.  En realidad las palabras en sí, no son de interés...el modelo requiere números no palabras. 

Lo triste es que no será posible leer las críticas...siquiera para tener una idea de si funciona el análiisis, o no.

Y, ¿cómo son las etiquetas (metas)?

In [30]:
y_train[0]

1

Son simplemente 0 ó 1, que indica sí al que escribió la crítica le gustó, o no, la película.

Para resumir, para el entrenamiento se tiene un conjunto de críticas de películas que han sido convertidas a vectores de palabras representadas por enteros, y una clasificación de sentimiento binaria.

Las RNNs pueden "explotar""
 (se habló de esto en clase) muy rápidamente.  Para que no se sobrecarguen las PCs que se podrían usar, se limitarán las críticas a las primeras 80 palabras:

In [31]:
X_train = sequence.pad_sequences(X_train, maxlen = 80)
X_test = sequence.pad_sequences(X_test, maxlen = 80)

## Configuración del modelo

Considerando lo complejo que es una RNN LSTM (debajo del "capó"), es increíble lo simple que resulta esta parte utilizando Keras.

Se empieza con una capa de incrustación "embedded".  La función de esta es convertir los datos de entrada a vectores densos de un tamaño fijo que son más adecuados para una red neuronal.  Esto es típico cuando se manejan indices de datos basados en texto.  El 20,000 representa el tamaño del vocabulario (límite impuesto para este ejercicio, pero puede variarse dependiendo de la capacidad de cómputo que se tiene) y 128 es la dimensión de 128 unidades de salida.

Luego se coloca una capa LSTM y ya!  Se especifica 128 para igualar la salida de la capa de incrustación, y se utiliza la opción de "botar" unidades para evitar el sobre ajuste, que es una característica muy común de las RNN.

Finalmente se debe reducir todo a una unidad de salida con una función de activación sigmoidal para seleccionar la clasificación binaria de sentimiento de 0 ó 1.

In [32]:
modelo = Sequential()
modelo.add(Embedding(50000, 128))
modelo.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
modelo.add(Dense(1, activation='sigmoid'))

Ya que este es un problema de clasificación binaria, la mejor función de pérdida es la "binary_crossentropy".  También se utiliza el optimizador "Adam" que es uno de los mejores.  Siempre es de recordar que si es necesario afinar más el modelo, se pueden probar otras.

In [33]:
modelo.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## Entrenamiento

Las RNNs, al igual que las CNNs son bastante pesadas en el uso de recursos.  Para poder utilizar una PC normal, el mantener el tamaño de las tandas pequeño, es clave.  En un mundo más profesional, se estaría sacando ventaja de GPUs instaladas en un cluster de computadoras para tener un mejor rendimiento.

## Advertencia

Esto puede tardarse bastante tiempo, aún en una PC potente!  A menos que se quiera ver todo el proceso y esperar una o más horas, no ejecute las siguientes celdas de código.

Esta siguiente celda es solo para quienes estén utilizando tensorflow-gpu con Windows.  Sirve para evitar problemas de asignación de memoria.

In [34]:
import tensorflow as tf
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # hacer que la memoria usada del GPU crezca dinamicamente 
sess = tf.compat.v1.Session(config = config)
set_session(sess)  # hacer que esta sesión de TensorFlow sea la sesión default de Keras

Ahora sí, a iniciar el entrenamiento.  Aún con una GPU se tardará bastante tiempo!

In [35]:
modelo.fit(X_train, y_train,
          batch_size = 32,
          epochs = 15,
          verbose = 2,
          validation_data = (X_test, y_test))

Epoch 1/15


782/782 - 108s - loss: 0.4241 - accuracy: 0.7995 - val_loss: 0.3660 - val_accuracy: 0.8334 - 108s/epoch - 139ms/step
Epoch 2/15
782/782 - 115s - loss: 0.2308 - accuracy: 0.9104 - val_loss: 0.4160 - val_accuracy: 0.8304 - 115s/epoch - 147ms/step
Epoch 3/15
782/782 - 111s - loss: 0.1291 - accuracy: 0.9516 - val_loss: 0.5280 - val_accuracy: 0.8080 - 111s/epoch - 142ms/step
Epoch 4/15
782/782 - 103s - loss: 0.0749 - accuracy: 0.9744 - val_loss: 0.5526 - val_accuracy: 0.8172 - 103s/epoch - 131ms/step
Epoch 5/15
782/782 - 136s - loss: 0.0511 - accuracy: 0.9824 - val_loss: 0.7497 - val_accuracy: 0.8071 - 136s/epoch - 173ms/step
Epoch 6/15
782/782 - 151s - loss: 0.0388 - accuracy: 0.9866 - val_loss: 0.8700 - val_accuracy: 0.8088 - 151s/epoch - 193ms/step
Epoch 7/15
782/782 - 153s - loss: 0.0269 - accuracy: 0.9912 - val_loss: 0.7923 - val_accuracy: 0.8170 - 153s/epoch - 195ms/step
Epoch 8/15
782/782 - 145s - loss: 0.0177 - accuracy: 0.9940 - val_loss: 1.0514 - val_accuracy: 0.8128 - 145s/epoch 

OK, ahora a evaluar la exactitud del modelo:

In [36]:
perdida, exactitud = modelo.evaluate(X_test, y_test,
                            batch_size = 32,
                            verbose = 2)
print('Pérdida de la Prueba:', perdida)
print('Exactitud de la Prueba (Test accuracy):', exactitud)

782/782 - 14s - loss: 1.2683 - accuracy: 0.7986 - 14s/epoch - 18ms/step
Pérdida de la Prueba: 1.2682605981826782
Exactitud de la Prueba (Test accuracy): 0.7986000180244446


Cerca del 80%, no está mal considerando que se limitó el ejercicio a las primeras 80 palabras de cada crítica.

Por otro lado - hay que pensar en lo que se hizo en este ejercicio!  Una red neuronal que puede leer críticas y deducir si al autor le gustó la película o no, basado en el texto.  Y el modelo toma en cuenta el contexto y la posición de cada palabra.

Lo mejor de todo es que armar el modelo solamente requirió de algunas líneas de código!  Es increíble lo que se puede hacer con Keras!!!

In [37]:
modelo.save("Anal_Sentimiento.h5")

c:\Users\Andrès DLR\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Ver algo de las críticas

Del enlace provisto arriba, se obtuvo el siguiente código que permite ver el texto del primero de los comentarios.  No sería dificil verlos todos si se quisiera.

In [38]:
# Use the default parameters to keras.datasets.imdb.load_data
start_char = 1
oov_char = 2
index_from = 3
# Retrieve the training sequences.
(X_train, _), _ = tf.keras.datasets.imdb.load_data(
    start_char=start_char, oov_char=oov_char, index_from=index_from
)
# Retrieve the word index file mapping words to indices
word_index = tf.keras.datasets.imdb.get_word_index()
# Reverse the word index to obtain a dict mapping indices to words
# And add `index_from` to indices to sync with `x_train`
inverted_word_index = dict(
    (i + index_from, word) for (word, i) in word_index.items()
)
# Update `inverted_word_index` to include `start_char` and `oov_char`
inverted_word_index[start_char] = "[START]"
inverted_word_index[oov_char] = "[OOV]"
# Decode the first sequence in the dataset
decoded_sequence = " ".join(inverted_word_index[i] for i in X_train[0])

In [39]:
decoded_sequence

"[START] this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and sh